<a href="https://colab.research.google.com/github/cimbelli/Surface-Heat-Islands/blob/main/temperatura_max_estiva_capoluoghi-regione_1.16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stima delle temperatura massima mensile nei capoluoghi di regione

In [2]:
import os
import ee
import pandas as pd
import json
import geopandas as gpd
import requests

ee.Authenticate()
ee.Initialize()
!pip install rasterstats
!pip install wget
import wget
from rasterstats import zonal_stats

# scegli quale statistica estrarre per sezione sulla base delle temperature massime estratte per pixel
statistiche = "min mean max median"
stat_map = "median"
class_intervals = [0, 20, 25, 30, 35, 40, 45, 50, 55, 60, 100]

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=uiCTN9xiY1iAogyOl2DWqQJWcFeVrlFXzzvkXKng1HQ&tc=7lOYuq3vD0wkupuJ8mdMT09RubPj1gXVl-W7xVS9_4U&cc=Fy7QvNz1i9LSuli-w9BWVdmAdce-indt3YYQq6OVVfk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkaCkFvGtoldFFzFI3nhVZ2vivlBymgn-PrGRXQ1QsbeDa_bo-my0s

Successfully saved authorization token.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl

In [3]:
anni = [2019, 2020, 2021, 2022, 2023]

In [4]:
import folium
from folium import plugins

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
    ).add_to(self)

def scarica(lyr, name, aoi):
    url = lyr.getDownloadUrl({
        'name': name[:-4],
        'bands': ['ST'],
        'region': aoi,
        'scale': 30,
        'format': 'GEO_TIFF'
    })
    response = requests.get(url)
    with open(name, 'wb') as fd:
        fd.write(response.content)
    return None

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True,
        show=False
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True,
        show=False
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True,
        show=False
    )
}

In [5]:
# carica lo shape dei capoluoghi di regione
nomecap = 'caporeg.geojson'
if not(os.path.isfile(nomecap)):
  #wget.download('https://github.com/cimbelli/Surface-Heat-Islands/raw/5006ef2e37091eedf421f65ed9cbf80c15ccbdfd/' + nomecap)
  wget.download('https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/' + nomecap)
g = open(nomecap)
cc = json.load(g)

gdf = gpd.read_file(nomecap)
centro = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs)
#gdf["lon"] = gdf.to_crs(epsg=3857).centroid.to_crs(gdf.crs) #gdf["geometry"].to_crs(4326).centroid.x
#gdf["lat"] = gdf["geometry"].centroid.y
gdf["lon"] = centro.x
gdf["lat"] = centro.y

In [8]:
for i in range(len(cc['features'])):           # for all the records
#for i in range(17,len(cc['features'])):

    reg = cc['features'][i]['properties']['COD_REG']
    reg1 = ('0' if reg < 10 else '') + str(reg)
    id1 = cc['features'][i]['properties']['PRO_COM']
    procom1 = '0' * (6-len(str(id1))) + str(id1)

    coords = cc['features'][i]['geometry']['coordinates']
    fc = ee.FeatureCollection(cc['features'])
    comune = cc['features'][i]['properties']['COMUNE']
    print('========',comune ,'========')

    nomefilesez = 'sez11_' + procom1 +'.geojson'
    if not(os.path.isfile(nomefilesez)):
      urlfilesez = 'https://raw.githubusercontent.com/cimbelli/Surface-Heat-Islands/main/sez11/' + nomefilesez
      wget.download(urlfilesez)
    g1 = open(nomefilesez)
    df_sez11 = gpd.read_file(g1)
    df_sez11['PRO_COM'] = df_sez11['PRO_COM'].astype(int)
    df_sez11['SEZ'] = df_sez11['SEZ'].astype(int)

    df_sez11[["P1","P14","P15","P28","P29"]] = df_sez11[["P1","P14","P15","P28","P29"]].fillna(0)
    df_sez11.rename(columns={'P1': 'Tot_residenti'}, inplace=True)#.astype('int')
    df_sez11['Tot_residenti'] = df_sez11['Tot_residenti'].astype('int')
    df_sez11['Bambini_0-9'] = (df_sez11['P14'].astype('int') + df_sez11['P15'].astype('int')).astype('int')
    df_sez11['Anziani_>70'] = (df_sez11['P28'].astype('int') + df_sez11['P29'].astype('int')).astype('int')

    lat = gdf[gdf['PRO_COM']== id1]['lat'].values[0]
    lon = gdf[gdf['PRO_COM']== id1]['lon'].values[0]

    minx = gdf[gdf['PRO_COM']== id1].geometry.bounds.minx.values[0]
    maxx = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxx.values[0]
    miny = gdf[gdf['PRO_COM']== id1].geometry.bounds.miny.values[0]
    maxy = gdf[gdf['PRO_COM']== id1].geometry.bounds.maxy.values[0]

    region = ee.Geometry.BBox(minx, miny, maxx, maxy)

    # Crea la mappa per il comune e in estate
    centro1 = df_sez11.to_crs(epsg=3857).centroid.to_crs(df_sez11.crs)
    m = folium.Map(location=(centro1.y[0],centro1.x[0] ), tiles="cartodb positron", zoom_start=12)

    for anno in anni:
        anno1 = str(anno)[-2:]
        print('---',anno,'---')

        try:
          del sdf, df_o
        except:
          pass

        nome_img = 'ST_' + comune + '_'+ str(anno) + '.tif'

        if not(os.path.isfile(nome_img)):
            L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(anno) + '-06-01', str(anno) + '-09-30')
            L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(str(anno) + '-06-01', str(anno) + '-09-30')
            bands = ['ST_B10', 'QA_PIXEL']
            bandName = ['ST', 'QA_PIXEL']
            filtered_L9 = L9.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
            filtered_L8 = L8.filter(ee.Filter.lt('CLOUD_COVER', 40)).select(bands, bandName)#.map(cloudMask)
            LandsatColl = filtered_L8.merge(filtered_L9)
            LandsatColl = LandsatColl.select('ST').filterBounds(region)

            if LandsatColl.size().getInfo() > 0:
              lst = LandsatColl.max().clip(fc)
              thermal = lst.select('ST.*').multiply(0.00341802).add(149.0).subtract(273.15)
              n_img = LandsatColl.size().getInfo()

              # zonal stats per sezioni
#              print('scarico immagine', comune, anno, "n° acquisizioni", n_img)

              scarica(thermal, nome_img, region)
            else:
              print('nessuna immagine nel periodo selezionato')
              continue
        else:
            print(nome_img, 'già presente')

# calcola le statistiche zonali per anno
        stats = zonal_stats(df_sez11, nome_img, stats=statistiche)
        sdf = pd.DataFrame(stats)
        sdf = sdf.round(2)
        sdf.rename(columns={'min':'min'+anno1,'max':'max'+anno1,'median':'median'+anno1,'mean':'mean'+anno1 }, inplace=True)
        sdf['n_img'+anno1] = n_img
        sdf['class'+anno1] = pd.cut(sdf[stat_map + anno1], bins=class_intervals, labels=False)
        df_sez11 = pd.concat([df_sez11, sdf], axis=1)
        stats.clear()

# esporta in Excel
    sdf2 = df_sez11.drop['COD_REG','PRO_COM','P14','P15','P28','P29']
    sdf3 = sdf2.drop['geometry']
    sdf3.to_excel(comune + '_stats.xlsx', index=False)


#        print("Create statistiche", comune, "estate", anno)


    folium.Choropleth(
        geo_data=sdf2,
        name ="Temperature SUPERFICIALI estive per sezione 2011 - " + comune,
        data=sdf3,
        #columns=["SEZ", stat_map,"Tot_residenti","Bambini_0-9","Anziani_>70"],
        key_on="feature.properties.SEZ",
        fill_color= 'RdYlGn_r',
        bins=[20, 25, 30, 35, 40, 45, 50, 55, 60, 100],
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Temperatura (" + stat_map + ") superficiale estiva per sezioni 2011",
    ).add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
    SuburbName = folium.features.GeoJson(sdf2, style_function=style_function,
        control=False, tooltip=folium.features.GeoJsonTooltip(
            #fields=['SEZ', stat_map, "Tot_residenti","Bambini_0-9","Anziani_>70"],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
        )
    )
    m.add_child(SuburbName)
    m.keep_in_front(SuburbName)
    basemaps['Google Satellite'].add_to(m)

    folium.LayerControl().add_to(m)
    #m.save(comune + '_' + str(anno) + '.html')

    #break


======== Milano ========
--- 2019 ---
ST_Milano_2019.tif già presente
--- 2020 ---
ST_Milano_2020.tif già presente
--- 2021 ---
--- 2022 ---
--- 2023 ---


InvalidIndexError: ignored

In [10]:
df_sez11


,COD_REG,PRO_COM,SEZ,Tot_residenti,P14,P15,P28,P29,geometry,Bambini_0-9,...,mean22,median22,n_img22,class22,min23,max23,mean23,median23,n_img23,class23
0,3.0,15146,5984,224,3,12,20,31,"MULTIPOLYGON (((9.20533 45.52227, 9.20576 45.5...",15,...,50.08,49.96,40,6.0,51.34,52.21,51.67,51.65,38,7.0
1,3.0,15146,1209,112,10,5,9,2,"MULTIPOLYGON (((9.22427 45.51616, 9.22447 45.5...",15,...,51.10,51.30,40,7.0,53.70,54.99,54.32,54.53,38,7.0
2,3.0,15146,1062,439,15,19,16,30,"MULTIPOLYGON (((9.22489 45.49963, 9.22420 45.4...",34,...,53.21,53.28,40,7.0,54.70,56.18,55.48,55.50,38,8.0
3,3.0,15146,5439,286,9,11,16,20,"MULTIPOLYGON (((9.18992 45.50156, 9.19070 45.5...",20,...,51.23,51.26,40,7.0,52.18,54.13,52.78,52.62,38,7.0
4,3.0,15146,748,372,24,13,19,38,"MULTIPOLYGON (((9.19610 45.49842, 9.19618 45.4...",37,...,49.11,49.06,40,6.0,50.96,51.54,51.24,51.21,38,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6080,3.0,15146,2316,10,1,0,0,0,"MULTIPOLYGON (((9.26795 45.47067, 9.26648 45.4...",1,...,50.67,50.54,40,7.0,49.01,51.18,50.12,50.16,38,7.0
6081,3.0,15146,3011,3,0,0,0,0,"MULTIPOLYGON (((9.19342 45.42019, 9.19366 45.4...",0,...,47.14,47.33,40,6.0,44.88,49.77,47.93,47.99,38,6.0
6082,3.0,15146,3017,20,2,3,1,1,"MULTIPOLYGON (((9.19304 45.41848, 9.19434 45.4...",5,...,44.78,44.70,40,5.0,44.78,47.14,46.08,46.00,38,6.0
6083,3.0,15146,3028,9,0,0,0,0,"MULTIPOLYGON (((9.18418 45.41583, 9.18747 45.4...",0,...,45.38,45.46,40,6.0,44.60,49.59,47.01,47.00,38,6.0


In [ ]:
# scaricare i file html e xlsx sul pc locale
from google.colab import files
import glob

#html_list = glob.glob('*.html')
xlsx_list = glob.glob('*.xlsx')

#for file_h in html_list:
#  files.download(file_h)

for file_x in xlsx_list:
  files.download(file_x)

In [ ]:
for file_x in xlsx_list:
  files.download(file_x)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>